In [1]:
import os
%pwd

'c:\\Users\\Sasu4\\SHIP_Classification_using_Resnet\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\Sasu4\\SHIP_Classification_using_Resnet'

In [88]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    epoch:int
    
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    freeze_all: bool
    freeze_till: int
    

In [68]:
from Ship_Classifier.constants import *
from Ship_Classifier.utils.common import read_yaml,create_directories

In [94]:
class PrepareBaseModel:
  def __init__(self, config):
        self.config = config
        self.model=None
        self.optimizer = None  # Initialize as needed
        self.loss_fn = None  # Initialize as needed

In [95]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        params = self.params
        
        #create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            freeze_all=config.get("freeze_all", True),  # Default to True if not specified
            freeze_till=config.get("freeze_till", None)
        )

        return prepare_base_model_config
    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
            epoch=self.params.EPOCHS
        )

        return prepare_callback_config

In [71]:
import os
import urllib.request as request
from zipfile import ZipFile
import torch 
import time
from torch.utils.tensorboard import SummaryWriter

In [96]:

  
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig,model, optimizer, loss):
        self.config = config
        self.model = model
        self.optimizer = optimizer
        self.loss = loss
   
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return torch.utils.tensorboard.SummaryWriter(log_dir=tb_running_log_dir)  # Adjust as necessary to return the TensorBoard logger

    @property
    def _create_ckpt_callbacks(self):
       checkpoint_dir = self.config.checkpoint_model_filepath
       checkpoint = torch.save({
            'epoch': self.config.epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'loss': self.loss
       }, checkpoint_dir)
       return checkpoint
    
    def get_tb_ckpt_callbacks(self):
        tb_callback = self._create_tb_callbacks
        ckpt_callback = self._create_ckpt_callbacks
        return tb_callback, ckpt_callback

In [99]:
from Ship_Classifier.components.prepare_base_model import PrepareBaseModel

from Ship_Classifier.components.prepare_base_model import PrepareBaseModelConfig
try:
    config = ConfigurationManager()

    #Prepare the base model configuration
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    model = prepare_base_model.get_model()
    optimizer = prepare_base_model.get_optimizer()
    loss = prepare_base_model.get_loss_fn()

    # Prepare the callback configuration
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(
        config=prepare_callbacks_config,
        model=model,
        optimizer=optimizer,
        loss=loss_fn
    )
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise Exception(f"An error occurred while preparing the model or callbacks: {str(e)}") from e

[2024-08-23 12:38:02,756: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-23 12:38:02,759: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-23 12:38:02,762: INFO: common: created directory at: artifacts]


Exception: An error occurred while preparing the model or callbacks: 'PrepareBaseModel' object has no attribute 'full_model'